In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Reading the Databases

We will use five datas frames with the reference years between 2017 and 2021. In the initial analyses, we noticed some differences in the number of questions that were used to collect data from the samples. In order to work with consistent data for the comparison of the previous period and the event of the Covid-19 pandemic, the common issues were filtered to continue the other procedures


In [ ]:
df_2017 = pd.read_csv('/DataFrames/OSMI_2017_Mental_Health_in_Tech_Survey.csv') 
df_2018 = pd.read_csv('/DataFrames/OSMI_2018_Mental_Health_in_Tech_Survey.csv') 
df_2019 = pd.read_csv('/DataFrames/OSMI_2019_Mental_Health_in_Tech_Survey.csv') 
df_2020 = pd.read_csv('/DataFrames/OSMI_2020_Mental_Health_in_Tech_Survey.csv') 
df_2021 = pd.read_csv('/DataFrames/OSMI_2021_Mental_Health_in_Tech_Survey.csv') 


FileNotFoundError: ignored

In [ ]:
print("In the 2017 DF, the total number of responses were:", df_2017.shape[0],".And the number of questions:",df_2017.shape[1])
print("In the 2018 DF, the total number of responses were:", df_2018.shape[0],".And the number of questions:",df_2018.shape[1])
print("In the 2019 DF, the total number of responses were:", df_2019.shape[0],".And the number of questions:",df_2019.shape[1])
print("In the 2020 DF, the total number of responses were:", df_2020.shape[0],".And the number of questions:",df_2020.shape[1])
print("In the 2021 DF, the total number of responses were:", df_2021.shape[0],".And the number of questions:",df_2021.shape[1])

In [ ]:
all_columns_no_filter = df_2017.columns
print(all_columns_no_filter)


In [ ]:
df_2017.head()

#Treating the Data

As we are working with a database with a high number of columns (questions), we decided to reduce the number of columns we will work with by selecting only those that appear in the original 5 databases.

In [ ]:
# Adding a year tag 
def add_year_column(df,year):
  df['year'] = year

In [ ]:
add_year_column(df_2017,2017)
add_year_column(df_2018,2018)
add_year_column(df_2019,2019)
add_year_column(df_2020,2020)
add_year_column(df_2021,2021)

### Filtering the columns
In addition, we apply a second filter to the columns. We are only selecting columns that have a fill above 80%. 

Some columns are the same for all databases, but have different tokens in each database. To deal with this, we had to remove these markers.

In [ ]:
# Only columns with 80% filled answers are valid
def filter_columns(df, p=0.2):
  valid_columns = df.isna().sum() <= p * df.shape[0] 
  return df.loc[:,valid_columns]

In [ ]:
df_2017 = filter_columns(df_2017)
df_2018 = filter_columns(df_2018)
df_2019 = filter_columns(df_2019)
df_2020 = filter_columns(df_2020)
df_2021 = filter_columns(df_2021)

In [ ]:
def correct_columns_names(df, rToken, sToken):
  df.columns = df.columns.str.replace(rToken, sToken)

In [ ]:
#arrumar as perguntas
correct_columns_names(df_2017, '*', '')
correct_columns_names(df_2017, '<strong>', '')
correct_columns_names(df_2017, '</strong>', '')

correct_columns_names(df_2018, '*', '')
correct_columns_names(df_2018, '<strong>', '')
correct_columns_names(df_2018, '</strong>', '')

correct_columns_names(df_2019, '*', '')
correct_columns_names(df_2019, '<strong>', '')
correct_columns_names(df_2019, '</strong>', '')

correct_columns_names(df_2020, '*', '')
correct_columns_names(df_2020, '<strong>', '')
correct_columns_names(df_2020, '</strong>', '')

correct_columns_names(df_2021, '*', '')
correct_columns_names(df_2021, '<strong>', '')
correct_columns_names(df_2021, '</strong>', '')

After all this process, we reduced the number of columns in our databases to 33.




In [ ]:
commun_columns = df_2017.columns.intersection(df_2018.columns)
commun_columns = df_2019.columns.intersection(commun_columns)
commun_columns = df_2020.columns.intersection(commun_columns)
commun_columns = df_2021.columns.intersection(commun_columns)
len(commun_columns)

###Merging the data frames

We will deal with two data frames, one pre-pandemic (***pre_df***) and one during the pandemic (**during_df**).

In [ ]:
pre_df = pd.concat([df_2017[commun_columns], df_2018[commun_columns]])
pre_df = pd.concat([pre_df, df_2019[commun_columns]])
during_df = pd.concat([df_2020[commun_columns], df_2021[commun_columns]])

In [ ]:
pre_df.info()

In [ ]:
during_df.info()

#Exploratory Analysis


###Viewing our Data

In [ ]:
print(pre_df.shape)
pre_df.head()



In [ ]:
print(during_df.shape)
during_df.head()


In [ ]:
pre_df.columns

## First observations

**Total age distribution**

In [ ]:
# Concatenating the 'What is your age?' columns from pre-pandemic and during-pandemic dataframes
age_data = pd.concat([pre_df['What is your age?'], during_df['What is your age?']])

# Converting age data to numeric values (assuming it contains numerical values)
age_data = pd.to_numeric(age_data, errors='coerce')

# Removing missing or invalid values
age_data = age_data.dropna()

# Filtering ages over or equal to 18
age_data = age_data[age_data >= 18]

# Defining the range of values with intervals of 10
bin_range = np.arange(0, min(age_data.max(), 100) + 5, 5)

# Plotting the histogram
plt.figure(figsize=(8, 4))
plt.hist(age_data, bins=bin_range, edgecolor='black')
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Total Age Distribution')
plt.xticks(bin_range)
plt.show()


**Age distribution by period**

In [ ]:
# Extracting the 'What is your age?' column from the pre-pandemic dataframe and from the during-pandemic dataframe
age_data_pre = pre_df['What is your age?']
age_data_dur = during_df['What is your age?']

# Converting age data to numeric values (assuming it contains numerical values)
age_data_pre = pd.to_numeric(age_data_pre, errors='coerce')
age_data_dur = pd.to_numeric(age_data_dur, errors='coerce')

# Removing missing or invalid values
age_data_pre = age_data_pre.dropna()
age_data_dur = age_data_dur.dropna()

# Filtering ages over or equal to 18
age_data_pre = age_data_pre[age_data_pre >= 18]
age_data_dur = age_data_dur[age_data_dur >= 18]

# Defining the range of values with intervals of 10
bin_range = np.arange(0, min(age_data_pre.max(), age_data_dur.max(), 100) + 5, 5)

# Creating subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plotting the histogram for pre-pandemic period
axes[0].hist(age_data_pre, bins=bin_range, edgecolor='black', color='salmon')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Count')
axes[0].set_title('Pre-Pandemic Age Distribution')
axes[0].set_xticks(bin_range)

# Plotting the histogram for during-pandemic period
axes[1].hist(age_data_dur, bins=bin_range, edgecolor='black', color='salmon')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Count')
axes[1].set_title('During-Pandemic Age Distribution')
axes[1].set_xticks(bin_range)

# Adjusting spacing between subplots
plt.tight_layout()

# Displaying the plot
plt.show()



### --------

In [ ]:
# Selecting the 'Do you currently have a mental health disorder?' and 'year' columns from the pre_df DataFrame
tmp = pre_df[['Do you currently have a mental health disorder?', 'year']]

# Grouping the data by 'year' and counting the occurrences of each answer for 'Do you currently have a mental health disorder?'
# Calculating the percentage of each answer using normalize=True
pre_mental_health = tmp.groupby('year')['Do you currently have a mental health disorder?'].value_counts(normalize=True).unstack()

# Creating a bar plot using the pre_mental_health DataFrame
pre_mental_health.plot(kind='bar')

# Setting the title and labels for the plot
plt.title('Do you currently have a mental health disorder?')
plt.ylabel('Percent')
plt.legend(title='Answer', loc='upper left')

# Setting the figure size
plt.figure(figsize=(150, 150))


In [ ]:
# Selecting the 'Do you currently have a mental health disorder?' and 'year' columns from the during_df DataFrame
tmp = during_df[['Do you currently have a mental health disorder?', 'year']]

# Grouping the data by 'year' and counting the occurrences of each answer for 'Do you currently have a mental health disorder?'
# Calculating the percentage of each answer using normalize=True
dur_mental_health = tmp.groupby('year' )['Do you currently have a mental health disorder?'].value_counts(normalize=True).unstack()
dur_mental_health.plot(kind='bar')

# Setting the title, labels and size of the plot
plt.title('Do you currently have a mental health disorder?')
plt.ylabel('Percent')
plt.legend(title='Answer', loc='upper left')
plt.figure(figsize=(150, 150))

In [ ]:
# Concatenating the pre_mental_health and dur_mental_health DataFrames
complete_mental_health = pd.concat([pre_mental_health, dur_mental_health])

# Creating a bar plot using the complete_mental_health DataFrame
complete_mental_health.plot(kind='bar')

# Setting the title, labels and size of the plot
plt.title('Do you currently have a mental health disorder?')
plt.ylabel('Percent')
plt.legend(title='Answer', loc='upper left')
plt.figure(figsize=(150, 150))

**Total gender distribution**

In [ ]:
# Extracting the "What is your gender?" column from the data frame
gender_data = pd.concat([pre_df['What is your gender?'], during_df['What is your gender?']])

# Cleaning and mapping gender values to standard categories
gender_data = gender_data.dropna().str.lower().str.strip()
gender_data = gender_data.replace(['female', 'f', 'woman', 'cis woman'], 'Female')
gender_data = gender_data.replace(['male', 'm', 'man', 'cis man'], 'Male')

# Counting the occurrences of each gender category
gender_counts = gender_data.value_counts()

# Determining the threshold for grouping other genders
threshold = 0.01      # Any gender category with an occurrence 
                      # below this threshold will be grouped 
                      # into the 'Other' category.

# Grouping other genders with low occurrence into 'Other' category
other_genders = gender_counts[gender_counts < threshold * gender_data.shape[0]].index
gender_counts['Other'] = gender_counts[other_genders].sum()
gender_counts = gender_counts.drop(other_genders)

# Plotting the pie chart
labels = ['Male', 'Female', 'Other']
plt.figure(figsize=(4, 4))
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%')
plt.title('Total Gender Distribution')
plt.axis('equal')
plt.legend(labels, loc="upper right")
plt.show()

**Gender distribution by period**

In [ ]:
# Extracting the "What is your gender?" column from the data frame for pre-pandemic and during-pandemic periods
pre_gender_data = pre_df['What is your gender?']
during_gender_data = during_df['What is your gender?']

# Cleaning and mapping gender values to standard categories for pre-pandemic and during-pandemic periods
pre_gender_data = pre_gender_data.dropna().str.lower().str.strip()
pre_gender_data = pre_gender_data.replace(['female', 'f', 'woman', 'cis woman'], 'Female')
pre_gender_data = pre_gender_data.replace(['male', 'm', 'man', 'cis man'], 'Male')

during_gender_data = during_gender_data.dropna().str.lower().str.strip()
during_gender_data = during_gender_data.replace(['female', 'f', 'woman', 'cis woman'], 'Female')
during_gender_data = during_gender_data.replace(['male', 'm', 'man', 'cis man'], 'Male')

# Grouping the gender data by period and counting the occurrences of each gender category
pre_gender_counts = pre_gender_data.value_counts()
during_gender_counts = during_gender_data.value_counts()

# Determining the threshold for grouping other genders
threshold = 0.01  

# Grouping other genders with low occurrence into 'Other' category for pre-pandemic period
pre_other_genders = pre_gender_counts[pre_gender_counts < threshold * pre_gender_data.shape[0]].index
pre_gender_counts['Other'] = pre_gender_counts[pre_other_genders].sum()
pre_gender_counts = pre_gender_counts.drop(pre_other_genders)

# Grouping other genders with low occurrence into 'Other' category for during-pandemic period
during_other_genders = during_gender_counts[during_gender_counts < threshold * during_gender_data.shape[0]].index
during_gender_counts['Other'] = during_gender_counts[during_other_genders].sum()
during_gender_counts = during_gender_counts.drop(during_other_genders)

labels = ['Male', 'Female', 'Other']

fig, axs = plt.subplots(2, 2, figsize=(8,8))

# Plotting the pie chart for pre-pandemic period
axs[0, 0].pie(pre_gender_counts, labels=pre_gender_counts.index, autopct='%1.1f%%',  colors=['salmon', 'gold', 'lightsteelblue'])
axs[0,0].set_title('Gender Distribution - Pre-Pandemic')
axs[0,0].axis('equal')
axs[0,0].legend(labels, loc="upper right")


axs[0,1].bar(labels, pre_gender_counts, label=pre_gender_counts.index, color = ['salmon', 'gold', 'lightsteelblue'])
axs[0,1].legend(loc='upper right')


# Plot the pie chart for during-pandemic period
axs[1,0].pie(during_gender_counts, labels=during_gender_counts.index, autopct='%1.1f%%', colors=['salmon', 'gold', 'lightsteelblue'])
axs[1,0].set_title('Gender Distribution - During Pandemic')
axs[1,0].axis('equal')
axs[1,0].legend(labels, loc="upper right")

axs[1,1].bar(labels, during_gender_counts, label=during_gender_counts.index, color = ['salmon', 'gold', 'lightsteelblue'])
axs[1,1].legend(loc='upper right')

plt.subplots_adjust(left=0.1, right=0.9, bottom=1.8, top=2, wspace=0.3, hspace=0.3)
plt.tight_layout()
plt.show()